In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import json

In [3]:
k_data = json.load(open("./KOK-VQA.json"))
k_df = pd.DataFrame(k_data["annotations"])

In [4]:
k_df["questions"]

0    [{'question_id': '1-1', 'question_ko': '사진속 어떤...
1    [{'question_id': '2-1', 'question_ko': '사진속 어떤...
2    [{'question_id': '1-1', 'question_ko': '이미지의 어...
Name: questions, dtype: object

In [5]:
k_df

,ACTION,CAPTION,HEIGHT,IMAGE_ACQUISITION_DATE,IMAGE_ACQUISITION_DATE,IMAGE_ID,IMAGE_NAME,IMAGE_URL,LICENSE,MAINOBJECT,MAINOBJECT_URL,SCENE,SUBOBJECT_1,SUBOBJECT_2,WIDTH,WIKI_FILE,bounding_box,questions
0,,,533,20181004.0,NaN,1,,https://upload.wikimedia.org/wikipedia/common...,FREE,piano,http://ko.dbpedia.org/resource/피아노,,Hands,Leg,800,Hupfeld piano played (2018-10-04 @pxhere 15460...,"[{'BOX_ID': '1-1', 'OBJECT': 'piano', 'X_COORD...","[{'question_id': '1-1', 'question_ko': '사진속 어떤..."
1,,,533,NaN,20181004,2,,https://upload.wikimedia.org/wikipedia/common...,FREE,pinpong,http://ko.dbpedia.org/resource/탁구,,person,table,800,Hupfeld piano played (2018-10-04 @pxhere 1546...,"[{'BOX_ID': '2-1', 'OBJECT': 'pingpong', 'X_CO...","[{'question_id': '2-1', 'question_ko': '사진속 어떤..."
2,,,533,20181004.0,NaN,3,,https://upload.wikimedia.org/wikipedia/common...,FREE,piano,http://ko.dbpedia.org/resource/피아노,,Hands,Leg,800,Hupfeld piano played (2018-10-04 @pxhere 15460...,"[{'BOX_ID': '1-1', 'OBJECT': 'piano', 'X_COORD...","[{'question_id': '1-1', 'question_ko': '이미지의 어..."


In [8]:
k_kb = json.load(open("./KOK-VQA-KB.json"))
kb_df = pd.DataFrame(k_kb["knowledgebase"])

JSONDecodeError: Expecting ',' delimiter: line 71 column 13 (char 2519)

In [ ]:
kb_df.T